### **PIP**

In [57]:
!pip install faiss-gpu

### ***Read***

In [58]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import os

In [59]:
df = pd.read_csv("/content/Income.csv")
def preprocess_data(df):
    chunks = []
    for index, row in df.iterrows():
        chunks.append(f"Row {index}: {row.to_dict()}")
    return chunks
chunks = preprocess_data(df)

In [60]:
print(chunks)

["Row 0: {'Name': 'Rob', 'Age': 27, 'Income(S)': 70000}", "Row 1: {'Name': 'Michael', 'Age': 29, 'Income(S)': 90000}", "Row 2: {'Name': 'Mohan', 'Age': 29, 'Income(S)': 61000}", "Row 3: {'Name': 'Ismail', 'Age': 28, 'Income(S)': 60000}", "Row 4: {'Name': 'Kory', 'Age': 42, 'Income(S)': 150000}", "Row 5: {'Name': 'Gautam', 'Age': 39, 'Income(S)': 155000}", "Row 6: {'Name': 'David', 'Age': 41, 'Income(S)': 160000}", "Row 7: {'Name': 'Andrea', 'Age': 38, 'Income(S)': 162000}", "Row 8: {'Name': 'Brad', 'Age': 36, 'Income(S)': 156000}", "Row 9: {'Name': 'Angelina', 'Age': 35, 'Income(S)': 130000}", "Row 10: {'Name': 'Donald', 'Age': 37, 'Income(S)': 137000}", "Row 11: {'Name': 'Tom', 'Age': 26, 'Income(S)': 45000}", "Row 12: {'Name': 'Arnold', 'Age': 27, 'Income(S)': 48000}", "Row 13: {'Name': 'Jared', 'Age': 28, 'Income(S)': 51000}", "Row 14: {'Name': 'Stark', 'Age': 29, 'Income(S)': 49500}", "Row 15: {'Name': 'Ranbir', 'Age': 32, 'Income(S)': 53000}", "Row 16: {'Name': 'Dipika', 'Age': 40

In [61]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)

In [37]:
# print(embeddings)

In [62]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [63]:
def retrieve_answer(question, k=1):
    question_embedding = model.encode([question])
    distances, indices = index.search(question_embedding, k)
    answers = [chunks[i] for i in indices[0]]
    return answers

In [73]:
question = "rob"
try:
    relevant_chunks = retrieve_answer(question)
    answer = f"Answer: {relevant_chunks}"
    print(answer)
except Exception as e:
    print(f"Error: {e}")

Answer: ["Row 0: {'Name': 'Rob', 'Age': 27, 'Income(S)': 70000}"]


In [72]:
import ast
key = 'Income(S)'
try:
    # Evaluate the string as a dictionary using ast.literal_eval to safely handle the dictionary-like format
    extracted_dict = ast.literal_eval(relevant_chunks[0].split(":", 1)[1].strip())
    # Now access the value using the key
    value = extracted_dict.get(key, "N/A")  # Use get() to avoid KeyError if key is not found
    print(f"{key}: {value}")
except (IndexError, SyntaxError, ValueError) as e:
    print(f"Error extracting data: {e}")

Income(S): 70000


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "EleutherAI/gpt-neo-125M"
# model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
input_text = "income"
inputs = tokenizer(answer, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_length=150,        # Increase max_length for a longer response
    temperature=0.8,       # Control randomness
    num_return_sequences=1,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id)  # Ensure the model knows when to stop
print(tokenizer.decode(outputs[0], skip_special_tokens=True))